<a href="https://colab.research.google.com/github/lamaachi/MyDataScience_Projects/blob/main/RNN_IMDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np

In [4]:
# -------------------------------------------------------------------------------
# 1. Chargement et Exploration du Dataset IMDB
# -------------------------------------------------------------------------------
# Le dataset IMDB contient 50 000 critiques de films étiquetées comme positives (1) ou négatives (0).
# Il est pré-traité pour inclure uniquement les mots les plus fréquents.

# Définir le nombre maximal de mots à considérer. Les mots moins fréquents seront ignorés.
# Choisir une valeur raisonnable permet de limiter la taille du vocabulaire et d'améliorer
# potentiellement les performances en se concentrant sur les mots les plus importants.
num_words = 10000

In [5]:
# Charger le dataset IMDB en utilisant la fonction fournie par Keras.
# `load_data` renvoie deux tuples : (x_train, y_train) et (x_test, y_test).
# `x_train` et `x_test` sont des listes de séquences, où chaque séquence représente une critique
# et est constituée d'indices de mots. `y_train` et `y_test` sont les étiquettes correspondantes.
# `num_words=num_words` limite le vocabulaire aux `num_words` mots les plus fréquents.
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=num_words)

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [6]:
# Afficher quelques informations sur les données chargées.
print("Nombre de séquences d'entraînement:", len(x_train))
print("Nombre de séquences de test:", len(x_test))
print("Exemple de séquence d'entraînement (indices de mots):", x_train[0])
print("Étiquette correspondante:", y_train[0])

Nombre de séquences d'entraînement: 25000
Nombre de séquences de test: 25000
Exemple de séquence d'entraînement (indices de mots): [1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 

In [27]:
# Pour comprendre le contenu des séquences, on peut décoder la première critique
# en mots lisibles en utilisant le dictionnaire de mapping des indices de mots.
word_index = imdb.get_word_index()
# print(word_index)
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
decoded_review = ' '.join([reverse_word_index.get(i - 3, '?') for i in x_train[0]])
print("Exemple de critique d'entraînement décodée:", decoded_review)

Exemple de critique d'entraînement décodée: ? this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert ? is an amazing actor and now the same being director ? father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for ? and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also ? to the two little boy's that played the ? of norman and paul they were just brilliant children are often left out of the ? list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be p

In [8]:
# -------------------------------------------------------------------------------
# 2. Pré-traitement des Données Séquentielles
# -------------------------------------------------------------------------------
# Les séquences dans le dataset IMDB ont des longueurs variables. Pour que notre modèle RNN
# puisse les traiter efficacement, nous devons les rendre de longueur uniforme.
# Nous allons utiliser le padding pour cela.

# Définir la longueur maximale souhaitée pour toutes les séquences.
# Les séquences plus longues seront tronquées, et les séquences plus courtes seront complétées (padding).
# Le choix de `maxlen` est crucial et peut influencer les performances du modèle.
# Une valeur trop petite peut entraîner une perte d'informations, tandis qu'une valeur trop grande
# peut augmenter la complexité du modèle et le temps d'entraînement.
maxlen = 200

In [9]:
# Utiliser la fonction `pad_sequences` de Keras pour uniformiser la longueur des séquences.
# `padding='post'` signifie que le padding sera ajouté à la fin des séquences.
# `truncating='post'` signifie que les séquences plus longues seront tronquées à la fin.
x_train_padded = sequence.pad_sequences(x_train, maxlen=maxlen, padding='post', truncating='post')
x_test_padded = sequence.pad_sequences(x_test, maxlen=maxlen, padding='post', truncating='post')

print("Séquence d'entraînement après padding (exemple):", x_train_padded[0])
print("Taille du tenseur des données d'entraînement après padding:", x_train_padded.shape)
print("Taille du tenseur des données de test après padding:", x_test_padded.shape)

Séquence d'entraînement après padding (exemple): [   1   14   22   16   43  530  973 1622 1385   65  458 4468   66 3941
    4  173   36  256    5   25  100   43  838  112   50  670    2    9
   35  480  284    5  150    4  172  112  167    2  336  385   39    4
  172 4536 1111   17  546   38   13  447    4  192   50   16    6  147
 2025   19   14   22    4 1920 4613  469    4   22   71   87   12   16
   43  530   38   76   15   13 1247    4   22   17  515   17   12   16
  626   18    2    5   62  386   12    8  316    8  106    5    4 2223
 5244   16  480   66 3785   33    4  130   12   16   38  619    5   25
  124   51   36  135   48   25 1415   33    6   22   12  215   28   77
   52    5   14  407   16   82    2    8    4  107  117 5952   15  256
    4    2    7 3766    5  723   36   71   43  530  476   26  400  317
   46    7    4    2 1029   13  104   88    4  381   15  297   98   32
 2071   56   26  141    6  194 7486   18    4  226   22   21  134  476
   26  480    5  144   30 55

In [20]:
# -------------------------------------------------------------------------------
# 3. Construction du Modèle RNN (LSTM)
# -------------------------------------------------------------------------------
# Nous allons construire un modèle RNN simple en utilisant une couche LSTM.
# LSTM (Long Short-Term Memory) est un type de RNN capable d'apprendre les dépendances à long terme,
# ce qui est crucial pour comprendre le sens des phrases dans les critiques de films.

# Initialiser un modèle séquentiel. Un modèle séquentiel est une pile linéaire de couches.
model = Sequential()

In [21]:
# Couche d'Embedding:
# Cette couche transforme les indices de mots en vecteurs denses de taille fixe (embeddings).
# `input_dim`: Taille du vocabulaire (nombre total de mots uniques).
# `output_dim`: Dimension de l'espace d'embedding (taille des vecteurs de mots).
# `input_length`: Longueur des séquences d'entrée (après padding).
embedding_dim = 128
model.add(Embedding(num_words, embedding_dim, input_length=maxlen))

In [22]:
# Couche LSTM:
# Une couche LSTM traite les séquences d'entrée et maintient un état interne (mémoire)
# pour capturer les informations séquentielles.
# `units`: Nombre d'unités de mémoire dans la couche LSTM (dimension de l'espace caché).
lstm_units = 128
model.add(LSTM(lstm_units))

In [23]:
# Couche Dense:
# Une couche dense (fully connected) pour la classification.
# `units`: Nombre de neurones dans la couche. Pour une classification binaire, on utilise 1 neurone.
# `activation='sigmoid'`: Fonction d'activation sigmoïde qui produit une probabilité entre 0 et 1,
# représentant la probabilité que la critique soit positive.
model.add(Dense(1, activation='sigmoid'))

In [24]:
# Afficher un résumé de l'architecture du modèle.
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [25]:
# -------------------------------------------------------------------------------
# 4. Compilation du Modèle
# -------------------------------------------------------------------------------
# Avant d'entraîner le modèle, nous devons le compiler en spécifiant l'optimiseur,
# la fonction de perte et les métriques à suivre.

# Optimiseur:
# `adam` est un optimiseur adaptatif couramment utilisé qui ajuste les taux d'apprentissage
# des différents paramètres du modèle pendant l'entraînement.
optimizer = 'adam'

In [26]:
# Fonction de Perte:
# `binary_crossentropy` est la fonction de perte appropriée pour les problèmes de classification binaire.
loss = 'binary_crossentropy'

In [28]:
# Métriques:
# `accuracy` est une métrique courante pour évaluer les performances d'un modèle de classification.
metrics = ['accuracy']

In [30]:
# Compiler le modèle en utilisant les configurations spécifiées.
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [44]:
# -------------------------------------------------------------------------------
# 5. Entraînement du Modèle
# -------------------------------------------------------------------------------
# Nous allons entraîner le modèle sur les données d'entraînement et évaluer ses performances
# sur les données de validation pendant l'entraînement.

# Définir le nombre d'époques (passages complets sur l'ensemble d'entraînement).
epochs = 20

In [45]:
# Définir la taille du lot (nombre d'échantillons traités à la fois pendant l'entraînement).
batch_size = 128

In [46]:
# Utiliser EarlyStopping pour arrêter l'entraînement si la performance sur l'ensemble de validation
# cesse de s'améliorer, ce qui aide à prévenir le surapprentissage.
# `monitor='val_loss'`: Surveiller la perte sur l'ensemble de validation.
# `patience=2`: Arrêter si la perte ne s'améliore pas pendant 2 époques consécutives.
early_stopping = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

In [51]:
# Diviser les données d'entraînement en un ensemble d'entraînement et un ensemble de validation.
# L'ensemble de validation est utilisé pour surveiller les performances du modèle pendant l'entraînement
# et pour aider à ajuster les hyperparamètres et prévenir le surapprentissage.
validation_split = 0.4

In [52]:
# Entraîner le modèle en utilisant les données d'entraînement et en validant sur une partie
# des données d'entraînement.
history = model.fit(x_train_padded, y_train,
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_split=validation_split,
                    callbacks=[early_stopping]
                    )

Epoch 1/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - accuracy: 0.9975 - loss: 0.0161 - val_accuracy: 0.9172 - val_loss: 0.4503
Epoch 2/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - accuracy: 0.9985 - loss: 0.0106 - val_accuracy: 0.9204 - val_loss: 0.4562
Epoch 3/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - accuracy: 0.9979 - loss: 0.0132 - val_accuracy: 0.9200 - val_loss: 0.4695


In [54]:
# -------------------------------------------------------------------------------
# 6. Évaluation du Modèle
# -------------------------------------------------------------------------------
# Après l'entraînement, nous évaluons les performances du modèle sur l'ensemble de test
# pour obtenir une estimation de sa capacité à généraliser à de nouvelles données.

# Évaluer le modèle sur les données de test.
loss, accuracy = model.evaluate(x_test_padded, y_test, verbose=0)
print("Perte sur l'ensemble de test:", loss)
print("Précision sur l'ensemble de test:", accuracy)

Perte sur l'ensemble de test: 0.9880833029747009
Précision sur l'ensemble de test: 0.8170400261878967


In [55]:
# -------------------------------------------------------------------------------
# 7. Faire des Prédictions (Optionnel)
# -------------------------------------------------------------------------------
# Une fois le modèle entraîné et évalué, nous pouvons l'utiliser pour faire des prédictions
# sur de nouvelles données.

# Faire des prédictions sur l'ensemble de test. Cela renverra des probabilités entre 0 et 1.
predictions = model.predict(x_test_padded)

782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step


In [39]:
# Pour obtenir des étiquettes binaires (0 ou 1), nous pouvons seuiller les probabilités.
# Par exemple, si la probabilité est supérieure à 0.5, on la considère comme une critique positive (1).
binary_predictions = (predictions > 0.5).astype(int)

In [40]:
# Afficher quelques prédictions.
print("Quelques prédictions (probabilités):", predictions[:10])
print("Quelques prédictions binaires:", binary_predictions[:10].flatten())
print("Vraies étiquettes pour ces exemples:", y_test[:10])

Quelques prédictions (probabilités): [[0.03688854]
 [0.9916578 ]
 [0.68579817]
 [0.2574341 ]
 [0.9482856 ]
 [0.80122745]
 [0.6805467 ]
 [0.05167335]
 [0.9411784 ]
 [0.94827366]]
Quelques prédictions binaires: [0 1 1 0 1 1 1 0 1 1]
Vraies étiquettes pour ces exemples: [0 1 1 0 1 1 1 0 0 1]


In [56]:
# -------------------------------------------------------------------------------
# 7.A. Fonction pour Faire une Prédiction sur une Nouvelle Phrase
# -------------------------------------------------------------------------------
"""
    Prédit le sentiment d'une phrase donnée en utilisant le modèle entraîné.

    Args:
        text (str): La phrase à analyser.

    Returns:
        str: "Positive" ou "Negative" indiquant le sentiment prédit.
"""
def predict_sentiment(text):

    # 1. Prétraitement de la nouvelle phrase:
    #    - Convertir la phrase en minuscules (bonne pratique pour l'uniformité).
    #    - Tokenisation: Séparer la phrase en mots.
    #      Ici, on utilise une simple division par les espaces, mais pour un traitement
    #      plus robuste, on pourrait utiliser un tokenizer plus avancé (comme celui de nltk ou spaCy).
    words = text.lower().split()

    #    - Conversion des mots en indices en utilisant le vocabulaire du dataset IMDB.
    #      Les mots qui ne sont pas dans le vocabulaire seront ignorés (ou mappés à un indice inconnu si géré).
    word_indices = [word_index.get(word, 0) + 3 for word in words]  # +3 car les indices 0, 1, 2 sont réservés

    #    - Padding ou troncature de la séquence pour correspondre à la longueur attendue par le modèle.
    padded_sequence = sequence.pad_sequences([word_indices], maxlen=maxlen, padding='post', truncating='post')

    # 2. Faire la prédiction avec le modèle:
    prediction = model.predict(padded_sequence)

    # 3. Interpréter la prédiction:
    #    - La sortie du modèle est une probabilité entre 0 et 1.
    #    - Un seuil (généralement 0.5) est utilisé pour classer le sentiment.
    if prediction[0] > 0.5:
        return "Positive"
    else:
        return "Negative"

In [59]:
# -------------------------------------------------------------------------------
# 7.B. Tester la Fonction de Prédiction
# -------------------------------------------------------------------------------

# Exemples de phrases à tester
positive_review = "This movie was absolutely fantastic! The acting, the plot, everything was perfect."
negative_review = "I was really disappointed by this film. The story was boring and the actors were unconvincing."
neutral_review = "The movie was okay, nothing special but not terrible either." # Exemple d'une phrase plus neutre

# Faire des prédictions et afficher les résultats
print(f"Sentiment de la phrase positive: {predict_sentiment(positive_review)}")
print(f"Sentiment de la phrase négative: {predict_sentiment(negative_review)}")
print(f"Sentiment de la phrase neutre: {predict_sentiment(neutral_review)}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Sentiment de la phrase positive: Positive
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Sentiment de la phrase négative: Negative
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Sentiment de la phrase neutre: Negative



# 8. Bonnes Pratiques comme un Data Scientist Senior


# 8.1. Exploration Approfondie des Données (EDA):
Avant même de commencer le prétraitement, un data scientist senior passerait
du temps à explorer le dataset. Cela inclut :
- Visualiser la distribution des longueurs des séquences pour prendre une
décision éclairée sur `maxlen`.
- Analyser la fréquence des mots pour comprendre le vocabulaire et potentiellement
identifier les mots à exclure ou à traiter spécialement.
- Examiner des exemples de critiques positives et négatives pour avoir une intuition
sur les caractéristiques linguistiques associées à chaque sentiment.

# 8.2. Expérimentation avec Différentes Architectures RNN:
Un SimpleRNN, un GRU (Gated Recurrent Unit) ou plusieurs couches LSTM pourraient
être essayés. Chaque architecture a ses propres forces et faiblesses. Un data
scientist senior testerait différentes configurations pour trouver celle qui
fonctionne le mieux pour ce problème spécifique.

# 8.3. Optimisation des Hyperparamètres:
Les hyperparamètres comme `embedding_dim`, `lstm_units`, `batch_size`, le taux
d'apprentissage de l'optimiseur, etc., ont un impact significatif sur les
performances du modèle. Un data scientist senior utiliserait des techniques
comme la validation croisée ou la recherche sur grille (Grid Search) ou la
recherche aléatoire (Random Search) pour trouver les valeurs optimales.

# 8.4. Utilisation d'un Ensemble de Validation Approprié:
S'assurer que l'ensemble de validation est représentatif des données d'entraînement
et qu'il n'y a pas de fuite de données entre les ensembles. L'utilisation de
`validation_split` est une méthode simple, mais pour des projets plus complexes,
une division stratifiée ou une validation croisée pourraient être nécessaires.

# 8.5. Techniques de Régularisation:
Pour lutter contre le surapprentissage, un data scientist senior utiliserait
des techniques de régularisation telles que :
- Dropout: Ignorer aléatoirement certains neurones pendant l'entraînement.
- L1 ou L2 Regularization: Ajouter une pénalité à la fonction de perte basée
sur la magnitude des poids du modèle.

# 8.6. Gestion du Vocabulaire et des Mots Hors Vocabulaire (OOV):
Décider comment traiter les mots qui n'apparaissent pas dans le vocabulaire
limité par `num_words`. Des techniques comme l'utilisation d'un jeton spécial
pour les mots OOV ou l'augmentation du vocabulaire pourraient être envisagées.

# 8.7. Analyse des Erreurs:
Après l'entraînement, examiner les exemples où le modèle s'est trompé. Cela
peut fournir des indices sur les aspects que le modèle a du mal à apprendre
et guider les améliorations futures.

# 8.8. Pipelines de Données et Reproductibilité:
Utiliser des pipelines de données pour organiser le prétraitement et s'assurer
de la reproductibilité des expériences. Cela implique de structurer le code
de manière modulaire et d'utiliser des outils de gestion des expériences.

# 8.9. Documentation et Commentaires:
Écrire un code clair, bien commenté et documenté est essentiel pour la collaboration
et la maintenance du projet.

# 8.10. Versionnement du Code et des Modèles:
Utiliser un système de contrôle de version (comme Git) pour suivre les modifications
du code et des modèles. Cela permet de revenir à des versions précédentes si nécessaire
et de collaborer efficacement.

En appliquant ces pratiques, un data scientist senior peut construire des modèles
plus robustes, plus performants et plus faciles à maintenir. Le code ci-dessus
fournit une base solide, mais l'exploration, l'expérimentation et l'itération
sont des étapes clés pour atteindre des résultats optimaux.